In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 定义一个简单的模型
class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 16, 3)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(16*30*30, 10)  # 假设输入是3x32x32

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = x.flatten(1)
        x = self.fc(x)
        return x

# 1. 初始化模型
model = SimpleModel()
# 2. 用torch.compile编译模型（核心步骤）
model = torch.compile(model)  # 一行代码完成编译

# 后续训练/推理代码完全不变
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# 模拟训练
for epoch in range(2):
    # 模拟输入：batch_size=8, channel=3, size=32x32
    inputs = torch.randn(8, 3, 32, 32)
    labels = torch.randint(0, 10, (8,))
    
    optimizer.zero_grad()
    outputs = model(inputs)  # 编译后的模型调用方式不变
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

# 推理时同样可用
with torch.no_grad():
    test_input = torch.randn(1, 3, 32, 32)
    pred = model(test_input)
    print(f"推理输出形状: {pred.shape}")

In [1]:
import torch
print(f"PyTorch版本: {torch.__version__}")

PyTorch版本: 2.10.0+cu128


In [4]:
import torch
import torch.nn as nn

class SimpleModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(3, 16, 3)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

model = SimpleModel().eval()
x = torch.randn(1, 3, 32, 32)

# 一键编译模型（自动做图优化+算子优化）
compiled_model = torch.compile(model)

# 第一次执行：编译+执行（有编译开销）
compiled_output = compiled_model(x)
# 后续执行：直接运行优化后的图（无编译开销）
compiled_output = compiled_model(x)

# 验证结果一致性
print(torch.allclose(model(x), compiled_output))  # True

False


In [9]:
import torch
import torch.nn as nn

# 定义一个简单的模型
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(10, 64)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

# 创建模型和输入
model = SimpleNet()
x = torch.randn(32, 10)  # batch_size=32, input_dim=10

# 使用 torch.compile 编译模型（默认使用 inductor 后端）
compiled_model = torch.compile(model)

# 首次运行会触发编译（可能稍慢），后续运行会更快
output = compiled_model(x)
print(output.shape)  # 应该输出 torch.Size([32, 1])

torch.Size([32, 1])
